[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nekrut/bda/blob/colab/lectures/lecture6.ipynb)

# Lecture 6: FASTQ and Quality Scores

[![XKCD927](https://imgs.xkcd.com/comics/standards_2x.png)](https://xkcd.com/927/)

> **💡 Attribution:** This material uses examples from notebooks developed by [Ben Langmead](https://www.langmead-lab.org/teaching.html)

> **📚 Python concepts in this lecture**
>
> This lecture applies concepts from **Think Python** (3rd edition):
>
> | Chapter | Concepts used in this lecture |
> |---------|------------------------------|
> | [Ch. 9 - Lists](https://greenteapress.com/thinkpython/html/thinkpython010.html) | Creating lists, `append()`, slicing `[1:]`, iteration with `for`, nested lists, list comprehensions |
> | [Ch. 10 - Dictionaries](https://greenteapress.com/thinkpython/html/thinkpython011.html) | Dictionary syntax `{'key': value}`, lists as dictionary values (used in pandas DataFrames) |
> | [Ch. 11 - Tuples](https://greenteapress.com/thinkpython/html/thinkpython012.html) | Creating tuples `(a, b, c)`, tuple unpacking `name, seq, qual = ...`, tuple indexing `read[2]` |
>
> Look for *(Ch. N)* annotations in the code explanations below.

# FASTQ

This notebook explores [FASTQ](https://en.wikipedia.org/wiki/FASTQ_format), the most common format for storing sequencing reads.

FASTA and FASTQ are rather similar, but FASTQ is almost always used for storing *sequencing reads* (with associated quality values), whereas FASTA is used for storing all kinds of DNA, RNA or protein sequences (without associated quality values).

## Basic format

Here's a single sequencing read in FASTQ format:

```
@ERR294379.100739024 HS24_09441:8:2203:17450:94030#42/1
AGGGAGTCCACAGCACAGTCCAGACTCCCACCAGTTCTGACGAAATGATGAGAGCTCAGAAGTAACAGTTGCTTTCAGTCCCATAAAAACAGTCCTACAA
+
BDDEEF?FGFFFHGFFHHGHGGHCH@GHHHGFAHEGFEHGEFGHCCGGGFEGFGFFDFFHBGDGFHGEFGHFGHGFGFFFEHGGFGGDGHGFEEHFFHGE
```

It's spread across four lines. The four lines are:

1. "`@`" followed by a read name
2. Nucleotide sequence
3. "`+`", possibly followed by some info, but ignored by virtually all tools
4. Quality sequence (explained below)

## Reading FASTQ with python

Download a sample small fastq file:

In [4]:
!curl -sLO https://zenodo.org/records/10602772/files/fastq_single_end_long.fq

#!curl -sLO https://zenodo.org/records/10602772/files/fastq_single_end_short.fq


Now we will use a very simple Python function to read this file and load fastq data into a list:

### Understanding the `parse_fastq()` function

> **📝 New Python concepts:** This function introduces **function definitions**, **while loops**, **string methods**, and **file handling**.
>
> 📚 **Think Python:** [Ch. 9 - Lists](https://greenteapress.com/thinkpython/html/thinkpython010.html) (lists, append) | [Ch. 11 - Tuples](https://greenteapress.com/thinkpython/html/thinkpython012.html) (tuples)

**Line-by-line breakdown:**

| Line | Code | Explanation |
|------|------|-------------|
| 1 | `def parse_fastq(fh):` | `def` creates a function named `parse_fastq`. `fh` is a **parameter**—a placeholder for whatever file handle we pass in when calling the function. |
| 3 | `reads = []` | Creates an empty list to store our results. *(Ch. 9: Lists)* |
| 4 | `while True:` | Starts an infinite loop. We'll use `break` to exit when done. |
| 5 | `first_line = fh.readline()` | `readline()` reads one line from the file and moves to the next. |
| 6 | `if len(first_line) == 0:` | When we reach end-of-file, `readline()` returns an empty string (length 0). |
| 7 | `break` | Exits the `while` loop immediately. |
| 8-9 | `if not first_line.startswith('@'):` | `startswith('@')` returns `True` if the string begins with `@`. If not, something's wrong with the file. |
| 10 | `name = first_line[1:].rstrip()` | `[1:]` is **slicing**—takes everything from position 1 onward (skips the `@`). `rstrip()` removes trailing whitespace/newlines. *(Ch. 9: Slicing)* |
| 11-13 | `seq = fh.readline().rstrip()` | Reads the sequence line, the `+` line (ignored), and quality line. |
| 14 | `reads.append((name, seq, qual))` | Creates a **tuple** `(name, seq, qual)` and adds it to our list. *(Ch. 9: append; Ch. 11: Tuples)* |
| 15 | `return reads` | Sends the completed list back to whoever called the function. |
| 17 | `with open(...) as fq:` | **Context manager**—opens file safely and auto-closes it when done. |
| 20 | `for read in reads:` | Loops through each tuple in the list. *(Ch. 9: Iteration)* |

In [8]:
def parse_fastq(fh):
    """ Parse reads from a FASTQ filehandle.  For each read, we
        return a name, nucleotide-string, quality-string triple. """
    reads = []
    while True:
        first_line = fh.readline()
        if len(first_line) == 0:
            break  # end of file
        if not first_line.startswith('@'):
            raise ValueError(f"Expected '@' at start of FASTQ record, got: {first_line[:20]!r}")
        name = first_line[1:].rstrip()
        seq = fh.readline().rstrip()
        fh.readline()  # ignore line starting with +
        qual = fh.readline().rstrip()
        reads.append((name, seq, qual))
    return reads

with open('fastq_single_end_long.fq','r') as fq:
    reads = parse_fastq(fq)

for read in reads:
    print(read)

('SRR7126301.1 1 length=102', 'GTTACCACAAATTACGATATCTTAGAGAAGAAAATGCTTAAACTAAATGGTTCATTGTTAGATCCTCAACTGTTTGGTAAGCCTAGACAGACTTTTTTATCC', '.GAG.G...<GGIGGGA.GAGGII..<<...<.<<A.AGAGAGG.<A.<...<GAGG.GG<.<.G.<GG.<A<.<<.<...<....<G..<.<GAA<A.<.G')
('SRR7126301.2 2 length=102', 'AGCCAGTGCAATATATACATGTATACTTAACAGATATGGAATGGTTGGCGAAGTAAATTTTTGGCGACGCGGTATGCGGAGTTGTAAGATGTACTACGGATC', 'IIIGGIGIIGIIIIIIIGGGGIGIGIGGGIGIIIGIIIIIIIIIGGIGGGIIIIIIIIIIGIGGGIGIGIIGGIIIIIIGIIIIGGGIIIIGGIIIIGIIIG')
('SRR7126301.3 3 length=102', 'ATTCCAATGTCTGAAAATAACGAAGAACAACATCAACAACAACAACAACAGCAACCTGTTGCCGTCGAAACTCCCTCGGCAGTTGAAGCTCCAGCATCAGCA', 'IIIIIIIIIIGIIGIIIIIIIIIGIIIGIIIIIIIIIIIIIIIIIGIIIIGIIIIIGIIIIIIIGGIIIIIGIIIIIIIIIGGGGGGIIIIGGGIIIIIGGI')
('SRR7126301.4 4 length=102', 'GATCCTGAGGCCTCACTAAGCCATTCAATCGGTACTAGCGACGGTCGGTGTGTACAAAGGGCAGGGACGTAATCAACGCAAGCTGATGACTTGCGCTTACTA', 'IIIIIGIIIGIIGGGGIIIGGGGIIIIIIIIIGIIGGGGGGIGAGGGG<AGGGGIIIIIGGGGGGIAGGGIIIIGIIIGIIGIIGGGGGGIGGGIGGIGIGG')
('SRR7126301.5 5 length=102', 'T

The nucleotide string can sometimes contain the character "`N`". `N` essentially means "no confidence." The sequencer knows there's a nucleotide there but doesn't know whether it's an A, C, G or T.

> **📝 A note on `while True`:** In Python, the while loop is used to repeatedly execute a block of code as long as a certain condition is true. The `while True` statement is a special case where the loop will run indefinitely until a `break` statement is encountered inside the loop. It is important to be careful when using while True loops, as they will run indefinitely if a break statement is not included. This can cause the program to crash or hang, if not handled properly.

> **💡 Production tip:** In real-world applications, use BioPython's `SeqIO.parse()` instead of writing your own parser. It handles edge cases and supports many file formats.

## Read name

Read names often contain information about:

1. The scientific study for which the read was sequenced. E.g. the string `ERR294379` (an [SRA accession number](http://www.ebi.ac.uk/ena/about/sra_format)) in the read names correspond to [this study](http://www.ncbi.nlm.nih.gov/sra/?term=ERR294379).
2. The sequencing instrument, and the exact *part* of the sequencing instrument, where the DNA was sequenced. See the [FASTQ format](http://en.wikipedia.org/wiki/FASTQ_format#Illumina_sequence_identifiers) Wikipedia article for specifics on how the Illumina software encodes this information.
3. Whether the read is part of a *paired-end read* and, if so, which end it is. The `/1` you see at the end of the read names above indicates the read is the first end from a paired-end read.

## Quality values

Quality values are probabilities. Each nucleotide in each sequencing read has an associated quality value. A nucleotide quality value encodes the probability that the nucleotide was *incorrectly called* by the sequencing instrument and its software. If the nucleotide is `A`, the corresponding quality value encodes the probability that the nucleotide at that position is actually *not* an `A`.

Quality values are encoded in two senses: first, the relevant probabilities are re-scaled using the Phred scale, which is a negative log scale.
In other words if *p* is the probability that the nucleotide was incorrectly called, we encode this as *Q* where *Q* = -10 * log10(*p*).

> **💡 Why Phred scale?** Error probabilities span a huge range (0.1 to 0.00001). The log transformation compresses this into manageable integers (10 to 50) that fit in a single ASCII character.

For example:
- If *Q* = 30, then *p* = 0.001, a 1-in-1000 chance that the nucleotide is wrong
- If *Q* = 20, then *p* = 0.01, a 1-in-100 chance
- If *Q* = 10, then *p* = 0.1, a 1-in-10 chance

Second, scaled quality values are *rounded* to the nearest integer and encoded using [ASCII printable characters](http://en.wikipedia.org/wiki/ASCII#ASCII_printable_characters). For example, using the Phred33 encoding (which is by far the most common), a *Q* of 30 is encoded as the ASCII character with code 33 + 30 = 63, which is "`?`". A *Q* of 20 is encoded as the ASCII character with code 33 + 20 = 53, which is "`5`".

Let's define some relevant Python functions:

### Understanding the quality conversion functions

These four functions convert between different representations of quality scores. Here's what each one does:

> **📝 New Python concepts:** **`ord()`** and **`chr()`** are built-in functions for working with ASCII characters. **Exponentiation** (`**`) raises a number to a power.

| Function | Purpose | Key code explained |
|----------|---------|-------------------|
| `phred33_to_q()` | Character → Q score | `ord(qual)` returns the ASCII code number for a character (e.g., `ord('?')` → 63). Subtract 33 to get Q. |
| `q_to_phred33()` | Q score → Character | `chr(Q + 33)` does the reverse—converts a number to its ASCII character (e.g., `chr(63)` → '?'). |
| `q_to_p()` | Q score → Probability | `10.0 ** (-0.1 * Q)` is the formula *p* = 10^(-Q/10). We use `10.0` (float) to ensure decimal division. |
| `p_to_q()` | Probability → Q score | `math.log10(p)` computes log base 10. `int(round(...))` rounds to nearest integer then converts to int. |

**Example walkthrough** for Q=30:
```
q_to_p(30) = 10.0 ** (-0.1 * 30) = 10.0 ** (-3) = 0.001
```
This means a 1-in-1000 chance the base call is wrong.

In [9]:
def phred33_to_q(qual):
    """ Turn Phred+33 ASCII-encoded quality into Phred-scaled integer """
    return ord(qual)-33

def q_to_phred33(Q):
    """ Turn Phred-scaled integer into Phred+33 ASCII-encoded quality """
    return chr(Q + 33)

def q_to_p(Q):
    """ Turn Phred-scaled integer into error probability """
    return 10.0 ** (-0.1 * Q)

def p_to_q(p):
    """ Turn error probability into Phred-scaled integer """
    import math
    return int(round(-10.0 * math.log10(p)))

In [10]:
# Here are the examples discussed above

# Convert Qs into ps
q_to_p(30), q_to_p(20), q_to_p(10)

(0.001, 0.01, 0.1)

In [11]:
p_to_q(0.00011) # note that the result is rounded

40

In [12]:
q_to_phred33(30), q_to_phred33(20)

('?', '5')

To convert an entire string of Phred33-encoded quality values into the corresponding *Q* or *p* values, we can do the following:

### Understanding this code block

> **📝 New Python concepts:** **`StringIO`** lets you treat a string as if it were a file. **Tuple unpacking** assigns multiple variables at once. **`map()`** applies a function to every element.
>
> 📚 **Think Python:** [Ch. 9 - Lists](https://greenteapress.com/thinkpython/html/thinkpython010.html) | [Ch. 11 - Tuples](https://greenteapress.com/thinkpython/html/thinkpython012.html) (tuple assignment)

| Line | Code | Explanation |
|------|------|-------------|
| 1 | `from io import StringIO` | Imports `StringIO` from the `io` module—lets us treat a string like a file. |
| 3-7 | `fastq_string = '''...'''` | Triple quotes create a multi-line string containing one FASTQ record. |
| 10 | `parse_fastq(StringIO(fastq_string))` | `StringIO(fastq_string)` wraps our string so `parse_fastq()` can read it like a file. |
| 10 | `...[0]` | Gets the first (and only) read from the returned list. *(Ch. 9: Indexing)* |
| 10 | `name, seq, qual = ...` | **Tuple unpacking**—the tuple `(name, seq, qual)` is split into 3 separate variables. *(Ch. 11: Tuple Assignment)* |
| 11 | `list(map(phred33_to_q, qual))` | `map()` applies `phred33_to_q` to each character in `qual`. `list()` converts the result to a list. |
| 12 | `list(map(q_to_p, q_string))` | Same pattern—applies `q_to_p` to each Q score. |

**Why `StringIO`?** Our `parse_fastq()` function expects a file handle (something it can call `.readline()` on). `StringIO` makes a string behave like a file, so we can test our parser without creating an actual file.

In [13]:
from io import StringIO

fastq_string = '''@ERR294379.100739024 HS24_09441:8:2203:17450:94030#42/1
AGGGAGTCCACAGCACAGTCCAGACTCCCACCAGTTCTGACGAAATGATG
+
BDDEEF?FGFFFHGFFHHGHGGHCH@GHHHGFAHEGFEHGEFGHCCGGGF
'''

# Take the first read from the small example above
name, seq, qual = parse_fastq(StringIO(fastq_string))[0]
q_string = list(map(phred33_to_q, qual))
p_string = list(map(q_to_p, q_string))
print(q_string)
print(p_string)

[33, 35, 35, 36, 36, 37, 30, 37, 38, 37, 37, 37, 39, 38, 37, 37, 39, 39, 38, 39, 38, 38, 39, 34, 39, 31, 38, 39, 39, 39, 38, 37, 32, 39, 36, 38, 37, 36, 39, 38, 36, 37, 38, 39, 34, 34, 38, 38, 38, 37]
[0.000501187233627272, 0.00031622776601683794, 0.00031622776601683794, 0.00025118864315095795, 0.00025118864315095795, 0.00019952623149688788, 0.001, 0.00019952623149688788, 0.00015848931924611126, 0.00019952623149688788, 0.00019952623149688788, 0.00019952623149688788, 0.0001258925411794166, 0.00015848931924611126, 0.00019952623149688788, 0.00019952623149688788, 0.0001258925411794166, 0.0001258925411794166, 0.00015848931924611126, 0.0001258925411794166, 0.00015848931924611126, 0.00015848931924611126, 0.0001258925411794166, 0.0003981071705534969, 0.0001258925411794166, 0.0007943282347242813, 0.00015848931924611126, 0.0001258925411794166, 0.0001258925411794166, 0.0001258925411794166, 0.00015848931924611126, 0.00019952623149688788, 0.000630957344480193, 0.0001258925411794166, 0.0002511886431

You might wonder how the sequencer and its software can *know* the probability that a nucleotide is incorrectly called. It can't; this number is just an estimate. To describe exactly how it's estimated is beyond the scope of this notebook; if you're interested, search for academic papers with "base calling" in the title. Here's a helpful [video by Rafa Irizarry](http://www.youtube.com/watch?v=eXkjlopwIH4).

A final note: other ways of encoding quality values were proposed and used in the past. For example, Phred64 uses an ASCII offset of 64 instead of 33, and Solexa64 uses "odds" instead of the probability *p*. But Phred33 is by far the most common today and you will likely never have to worry about this.

> **📝 A note on `map()`:** In Python, the `map()` function is used to apply a given function to all elements of an iterable (such as a list, tuple, or string) and return an iterator that yields the results. The `map()` function takes two arguments: a function that is to be applied to each element of the iterable, and an iterable on which the function is to be applied. Example: `squared_numbers = map(lambda x: x**2, [1, 2, 3, 4, 5])` returns `[1, 4, 9, 16, 25]`.

## How good are my reads?

Let's analyze the quality distribution across all reads in our FASTQ file. First, reload the reads:

In [14]:
# Reload reads from file
with open('fastq_single_end_short.fq','r') as fq:
    reads = parse_fastq(fq)

### Understanding nested list comprehensions

The next cell uses a **nested list comprehension**—a compact way to create a list of lists. Let's break it down:

> **📝 New Python concept:** **List comprehensions** are a concise way to create lists. The syntax `[expression for item in iterable]` creates a new list by applying `expression` to each `item`.
>
> 📚 **Think Python:** [Ch. 9 - Lists](https://greenteapress.com/thinkpython/html/thinkpython010.html) | [Ch. 11 - Tuples](https://greenteapress.com/thinkpython/html/thinkpython012.html) (tuple indexing)

**The code:** `[[phred33_to_q(q) for q in read[2]] for read in reads]`

**Step-by-step breakdown:**

1. **Outer loop**: `for read in reads` — iterates through each read tuple *(Ch. 9: Iteration)*
2. **Access quality string**: `read[2]` — gets the 3rd element (quality string) from the tuple *(Ch. 11: Tuple indexing)*
3. **Inner loop**: `for q in read[2]` — iterates through each character in the quality string
4. **Transform**: `phred33_to_q(q)` — converts each character to a Q score

**Equivalent using regular loops:**
```python
run_qualities = []
for read in reads:                    # outer loop
    qual_scores = []
    for q in read[2]:                 # inner loop over quality string
        qual_scores.append(phred33_to_q(q))
    run_qualities.append(qual_scores)
```

**Result structure:** A list of lists, where each inner list contains the Q scores for one read. *(Ch. 9: Nested lists)*

In [15]:
# Extract qualities and convert to numerical values using list comprehension
run_qualities = [[phred33_to_q(q) for q in read[2]] for read in reads]

### Understanding NumPy arrays and transpose

We need to [transpose](https://en.wikipedia.org/wiki/Transpose) this matrix so that instead of rows = reads and columns = positions, we get rows = positions and columns = reads. This lets us compute statistics *per position* across all reads:

```
Before (rows=reads):     After (rows=positions):
Read1: [Q1, Q2, Q3]      Pos1: [Q1_r1, Q1_r2, Q1_r3]
Read2: [Q1, Q2, Q3]  =>  Pos2: [Q2_r1, Q2_r2, Q2_r3]
Read3: [Q1, Q2, Q3]      Pos3: [Q3_r1, Q3_r2, Q3_r3]
```

> **📝 New Python concepts:** **NumPy** is a library for numerical computing. **`np.array()`** converts lists to arrays. **`.T`** transposes (swaps rows and columns).

| Code | Explanation |
|------|-------------|
| `import numpy as np` | Imports NumPy library, conventionally aliased as `np`. |
| `np.array(run_qualities)` | Converts our list of lists into a NumPy array—a grid-like structure optimized for math operations. |
| `.T` | The **transpose** attribute—swaps rows and columns. Equivalent to `np.transpose(array)`. |

**Why NumPy?** Python lists are flexible but slow for math. NumPy arrays are optimized for numerical operations and can compute statistics across entire rows/columns efficiently.

In [16]:
# Transpose using numpy
import numpy as np
base_qualities = np.array(run_qualities).T

### Understanding pandas DataFrames

Now we'll compute summary statistics for each position and store them in a **pandas DataFrame**—a table-like structure perfect for data analysis.

> **📝 New Python concepts:** **pandas** is a data analysis library. **`pd.DataFrame()`** creates a table from a dictionary. **`range()`** generates a sequence of numbers.
>
> 📚 **Think Python:** [Ch. 9 - Lists](https://greenteapress.com/thinkpython/html/thinkpython010.html) (list comprehensions) | [Ch. 10 - Dictionaries](https://greenteapress.com/thinkpython/html/thinkpython011.html) (dict syntax)

| Code | Explanation |
|------|-------------|
| `import pandas as pd` | Imports pandas, conventionally aliased as `pd`. |
| `pd.DataFrame({...})` | Creates a table where each key becomes a column name and each value becomes column data. *(Ch. 10: Dictionaries)* |
| `range(len(base_qualities))` | Generates numbers 0, 1, 2, ... up to the number of positions. These become row indices. |
| `[np.mean(b) for b in base_qualities]` | List comprehension that computes the mean of each row (position) in our transposed array. *(Ch. 9: Lists)* |
| `np.quantile(b, .25)` | Computes the 25th percentile—the value below which 25% of data falls. |

**NumPy statistics functions:**

| Function | What it computes |
|----------|-----------------|
| `np.mean(b)` | Average (sum divided by count) |
| `np.median(b)` | Middle value when sorted |
| `np.quantile(b, .25)` | 25th percentile |
| `np.quantile(b, .75)` | 75th percentile |
| `np.min(b)`, `np.max(b)` | Smallest and largest values |

In [18]:
# Compute per-position statistics using pandas
import pandas as pd

stats_df = pd.DataFrame({
    'base': range(len(base_qualities)),
    'mean': [np.mean(b) for b in base_qualities],
    'median': [np.median(b) for b in base_qualities],
    'q25': [np.quantile(b, .25) for b in base_qualities],
    'q75': [np.quantile(b, .75) for b in base_qualities],
    'min': [np.min(b) for b in base_qualities],
    'max': [np.max(b) for b in base_qualities]
})

In [ ]:
# This code block computes per-position quality statistics from your `base_qualities` array.
# `base_qualities` contains Phred quality scores for each base across all reads.

import pandas as pd

stats_df = pd.DataFrame({
    # 'base' column represents the position in the read.
    'base': range(len(base_qualities)),
    # 'mean' column calculates the average quality score for each position.
    'mean': [np.mean(b) for b in base_qualities],
    # 'median' column calculates the median quality score for each position.
    'median': [np.median(b) for b in base_qualities],
    # 'q25' column calculates the 25th percentile (first quartile) of quality scores for each position.
    'q25': [np.quantile(b, .25) for b in base_qualities],
    # 'q75' column calculates the 75th percentile (third quartile) of quality scores for each position.
    'q75': [np.quantile(b, .75) for b in base_qualities],
    # 'min' column calculates the minimum quality score for each position.
    'min': [np.min(b) for b in base_qualities],
    # 'max' column calculates the maximum quality score for each position.
    'max': [np.max(b) for b in base_qualities]
})
# The resulting `stats_df` DataFrame provides a summary of the quality distribution across all reads for each base position.

In [19]:
stats_df

,base,mean,median,q25,q75,min,max
0,0,32.666667,33.0,32.0,33.5,31,34
1,1,33.666667,35.0,33.0,35.0,31,35
2,2,33.000000,32.0,32.0,33.5,32,35
3,3,37.333333,37.0,36.5,38.0,36,39
4,4,33.333333,36.0,31.5,36.5,27,37
5,5,37.000000,37.0,37.0,37.0,37,37
6,6,32.333333,30.0,30.0,33.5,30,37
7,7,36.000000,36.0,35.5,36.5,35,37
8,8,36.666667,36.0,36.0,37.0,36,38
9,9,37.000000,37.0,36.0,38.0,35,39


### Understanding Altair visualization

The plot below shows quality distribution per position—similar to FastQC output. Each position shows:
- **Black line**: min to max range
- **Green bar**: interquartile range (25th to 75th percentile)  
- **Red tick**: median quality

> **📝 New Python concepts:** **Altair** is a declarative visualization library. You describe *what* you want to show, not *how* to draw it. Charts are built by chaining methods.

| Code | Explanation |
|------|-------------|
| `import altair as alt` | Imports Altair, conventionally aliased as `alt`. |
| `alt.Chart(stats_df)` | Creates a chart object from our DataFrame. |
| `.encode(alt.X('base:Q', ...))` | Maps the `base` column to the x-axis. `:Q` means "quantitative" (numeric). |
| `.properties(width=800, height=200)` | Sets the chart dimensions in pixels. |
| `.mark_tick()` | Draws small horizontal ticks (for median). |
| `.mark_rule()` | Draws lines/bars (for ranges). |
| `alt.Y('median:Q')` | Maps `median` column to y-axis. |
| `alt.Y2('q75:Q')` | Sets the end point for a range (used with `mark_rule`). |
| `median + q + min_max` | The `+` operator **layers** multiple charts on top of each other. |

**How layering works:**
```
min_max (black lines)     ─────────────────
         +
q (green bars)            ████████████████
         +                       
median (red ticks)              ─
         =
Combined plot             ─────█████─█████─────
```

In [29]:
# Plot quality per position
import altair as alt

base = alt.Chart(stats_df).encode(
    alt.X('base:Q', title="Position in the read")
).properties(
    width=900,
    height=200)

median = base.mark_tick(color='red',orient='horizontal').encode(
    alt.Y('median:Q', title="Phred quality Q-score"),
)

q = base.mark_rule(color='green',opacity=.5,strokeWidth=10).encode(
    alt.Y('q25:Q'),
    alt.Y2('q75:Q')
)

min_max = base.mark_rule(color='black').encode(
        alt.Y('min:Q'),
        alt.Y2('max:Q')
)

median + q + min_max

alt.LayerChart(...)

## Other comments

In all the examples above, the reads in the FASTQ file are all the same length. This is not necessarily the case though it is usually true for datasets generated by sequencing-by-synthesis instruments. FASTQ files can contain reads of various lengths.

FASTQ files often have the extension `.fastq` or `.fq`.